In [ ]:
import pandas as pd


url = "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_original/lyric_word_parsed_by_line.csv"
words_df = pd.read_csv(url, encoding="utf-8")


In [ ]:
song_count_df = (
    words_df.groupby(["year"])
    .nunique(dropna=False)[["song_id"]]
    .rename(columns={"song_id": "song_count_by_year"})
)
song_count_df


In [ ]:
words_with_song_count_df = pd.merge(words_df, song_count_df, on="year", how="inner")

words_by_year_df = (
    words_with_song_count_df.groupby(["word", "tag_orig", "year"])
    .agg(
        count=("word", "count"),
        song_count_by_year=("song_count_by_year", "last"),
    )
    .reset_index()
)

words_by_year_df["ratio"] = (
    words_by_year_df["count"] / words_by_year_df["song_count_by_year"]
)

period = 2020 - 1964 + 1


def calc_mean_ratio(x):
    return x.sum() / period


words_by_year_df["mean_ratio"] = words_by_year_df.groupby(["word", "tag_orig"])[
    "ratio"
].transform(calc_mean_ratio)
words_by_year_df = words_by_year_df.sort_values("mean_ratio", ascending=False)
words_by_year_df


In [ ]:
top_100_words_df = (
    words_by_year_df.groupby(["word", "tag_orig"])
    .agg(mean_ratio=("mean_ratio", "last"))
    .reset_index()
    .sort_values("mean_ratio", ascending=False)
)
top_100_words_df = (
    top_100_words_df.loc[top_100_words_df["tag_orig"] == "NNG"]
    .nlargest(columns="mean_ratio", n=100)
    .reset_index()[["word", "tag_orig", "mean_ratio"]]
)
top_100_words_df["rank"] = top_100_words_df["mean_ratio"].rank(ascending=False)
top_100_words_df


In [ ]:
words_by_year_top_100_df = pd.merge(
    top_100_words_df, words_by_year_df, on=("word", "tag_orig"), how="inner"
)[["word", "rank", "year", "song_count_by_year", "ratio", "mean_ratio_x"]].sort_values(
    ["year", "rank"], ascending=True
)[
    ["year", "word", "ratio"]
]
words_by_year_top_100_df = words_by_year_top_100_df.pivot_table(
    words_by_year_top_100_df, index="year", columns=words_by_year_top_100_df["word"]
).fillna(0)


In [ ]:
words_by_year_top_100_df.to_csv(
    "../data/lyrics_count/lyric_word_counted_top_100_NORM.csv"
)
